# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing) и затем содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [1]:
import itertools
import numpy as np

In [79]:
def solve_bp_decision(weights, n_bins):
    weights = np.array(weights)
    for w in weights:
        if w > 1:
            return False
    rng = np.arange(0, len(weights))
    for index in itertools.permutations(rng):
        cur_weight = weights[list(index)]
        now_w = 0
        bin_used = 1
        for w in cur_weight:
            if now_w + w <= 1:
                now_w += w
                continue
            else:
                now_w = w
                bin_used += 1
            if bin_used > n_bins:
                bin_used = n_bins + 1
                break
        if bin_used <= n_bins:
            return True
    return False


In [80]:
def solve_bp_evaluation(weights):
    l, r = 0, len(weights)
    while l + 1 != r:
        mid = (l + r) // 2
        if solve_bp_decision(weights=weights, n_bins=mid):
            r = mid
        else:
            l = mid
    return r

In [81]:
def getNewWeights(old_weights, bins):
    new_weights = np.zeros(bins.max() + 1, dtype=float)
    flag = False
    for i in range(len(old_weights)):
        new_weights[bins[i]] += old_weights[i]
        if new_weights[bins[i]] > 1:
            flag = True
    return not flag, new_weights

def solve_bp_search(weights):
    weights = np.array(weights)
    solution = np.arange(weights.shape[0])
    best_n_bins = solve_bp_evaluation(weights)
    now_bin = 0
    while True:
        for i in range(solution.shape[0]):
            if solution[i] <= now_bin:
                continue
            buffer = solution[i]    
            solution[i] = now_bin
            can, new_weights = getNewWeights(weights, solution)
            if can and solve_bp_evaluation(new_weights) == best_n_bins:
                continue
            solution[i] = buffer
        now_bin += 1
        if now_bin > best_n_bins:
            break
    return solution + 1

Немного протестируем

In [92]:
print(solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2))
print(solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3))

pack = [0.6, 0.6, 0.4, 0.4]
print('\ntest\'s for', pack, 'pack')
for i in range(4):
    print('n_bins =', i, ' : ', solve_bp_decision(pack, i))
    
print(solve_bp_evaluation(pack))

print('pack :', pack, 'solution :', solve_bp_search(pack), sep='\n')

False
True

test's for [0.6, 0.6, 0.4, 0.4] pack
n_bins = 0  :  False
n_bins = 1  :  False
n_bins = 2  :  True
n_bins = 3  :  True
2
pack :
[0.6, 0.6, 0.4, 0.4]
solution :
[1 2 1 2]
